In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from string import Template
import os
import openai
import nltk
import requests
import ast
from conversations import BotQuestion, ConvMode, Script
import json
import requests
from typing import List

openai.api_key = os.environ["OPENAI_API_KEY"]


## HELPER FUNCTIONS

In [2]:
def merge_dicts(dict1, dict2):
    """menggabungkan dua dictionaries dan keynya di combine jadi integer"""
    merged_dict = {}
    i = 1
    for key in sorted(dict1.keys(), key=int):
        merged_dict[i] = dict1[key]
        i += 1

    for key in sorted(dict2.keys(), key=int):
        merged_dict[i] = dict2[key]
        i += 1

    return merged_dict

## CUSTOM PROMPT

In [3]:
prompt = Template("""
    Jawab secara singkat, \n 
    
    $pertanyaan 
    
    buat jawaban dalam bentuk JSON seperti contoh ini:
    
    {
    'pertanyaan' : '..',
    'jawaban' : '..'
    }
    
    tampilkan hanya JSON saja.
    """)


In [4]:
kebutuhan = """
administrative function.
Positive working attitude.
Good communication skills
Self motivated and able to work independently with minimum supervision.
Computer literate and good command of written and spoken English, Chinese and Malay
Pleasant & polite, with initiative and commited with strong sense of responsibility
Basic Health knowledge will be an added advantage
"""

kebutuhan_kompetensi = kebutuhan.split('\n')
kebutuhan_kompetensi = [x for x in kebutuhan_kompetensi if x != ""]
print(kebutuhan_kompetensi)

['administrative function.', 'Positive working attitude.', 'Good communication skills', 'Self motivated and able to work independently with minimum supervision.', 'Computer literate and good command of written and spoken English, Chinese and Malay', 'Pleasant & polite, with initiative and commited with strong sense of responsibility', 'Basic Health knowledge will be an added advantage']


In [5]:
template_buat_pertanyaan = Template("""
gpt Kamu adalah pewawancara pencari kerja yang sangat teliti. 
Buatkan Beberapa Pertanyaan dalam bahasa indonesia untuk menggali pelamar kerja tentang hal-hal seputar kompetensi dibawah ini:

Kebutuhan Kompetensi: $kebutuhan_kompetensi

Kamu akan menjawab dalam format JSON dalam bentuk seperti contoh ini:
    
{
    '1' : 'Pertanyaan 1 ..',
    '2' : 'Pertanyaan 2 ..',
    '3' : 'Pertanyaan 3 ..',
    '4' : 'Pertanyaan 4 ..',
}

Berikan jawaban kamu hanya dalam bentuk JSON saja, tanpa menggunakan keterangan.
""")


In [6]:
tmpl = template_buat_pertanyaan.substitute({'kebutuhan_kompetensi' : kebutuhan_kompetensi[2]})

In [7]:
from dataclasses import dataclass
from string import Template

@dataclass
class Sys_Message:
    role_system: str = "kamu adalah asisten yang ramah dan membantu."
    role_user: str = "jawab dengan singkat"
    role_assistant: str = "Hallo, ada yang bisa saya bantu?"


def custom_gpt(sys_msg: Sys_Message, prompt: str) -> str:
    
    messages = []
    messages.append({'role' : 'system', 'content' : sys_msg.role_system})
    messages.append({'role' : 'user', 'content' : sys_msg.role_user})
    messages.append({'role' : 'assistant' , 'content' : sys_msg.role_assistant})   
    messages.append({'role': 'user', 'content' : prompt})
    
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
#      max_tokens=1024,
#      n=1,
#      stop=None,
      temperature=0.3,
    )

    message = response.choices[0].message.content
    messages.append({"role" : "assistant", "content" : message})
    #print(messages)
    return message

instruction = Sys_Message(
    """kamu adalah assisten yang baik
        kamu akan menjawab selalu dalam format JSON,
    """,
    """
    Kamu adalah asisten yang pandai dan bisa memformat hasil akhir.
    """    
    ,
    "Hallo, nama saya Maya",
)

#jawaban = ast.literal_eval(custom_gpt(instruction, tmpl))
#print(jawaban)
#kumpulan_pertanyaan = merge_dicts(kumpulan_pertanyaan, jawaban)

In [8]:
#kumpulan_pertanyaan

 5: 'Seberapa familiar Anda dengan konsep kesehatan dasar?',
 6: 'Bagaimana Anda memperoleh pengetahuan tentang kesehatan dasar?',
 7: 'Dapatkah Anda memberikan contoh situasi di mana pengetahuan kesehatan dasar sangat diperlukan?',
 8: 'Bagaimana Anda berencana untuk meningkatkan pengetahuan Anda tentang kesehatan dasar jika Anda diterima di perusahaan ini?',
 9: 'Bagaimana kamu biasanya berkomunikasi dengan rekan kerja atau atasanmu?',
 10: 'Bagaimana cara kamu menangani situasi di mana kamu harus berkomunikasi dengan seseorang yang sulit diajak berbicara?',
 11: 'Bagaimana kamu memastikan bahwa pesan yang kamu sampaikan kepada orang lain dipahami dengan baik?',
 12: 'Bagaimana kamu menangani situasi di mana ada kesalahpahaman dalam komunikasi dengan rekan kerja atau atasanmu?'

## FORCED

In [9]:
kumpulan_pertanyaan = {0: 'Bagaimana cara Anda memotivasi diri sendiri ketika bekerja secara mandiri?',
 1: 'Apakah Anda pernah bekerja secara mandiri sebelumnya? Jika ya, ceritakan pengalaman Anda dan bagaimana Anda menghadapi tantangan tersebut.',
 2: 'Bagaimana Anda mengatur waktu dan prioritas ketika bekerja secara mandiri?',
 3: 'Bagaimana Anda menyelesaikan tugas yang sulit atau kompleks ketika tidak ada orang lain yang bisa membantu?',
}

## PUT QUESTION TO BOT

In [10]:
def make_botquestion(all_question: dict) -> List[BotQuestion]:
    result = []
    for key,value in all_question.items():
        result.append(BotQuestion(id=key, question=value))
    return result



botquestion = make_botquestion(kumpulan_pertanyaan)
user_number = "62895352277562@c.us"

# 62895352277562
# 62895391400811 tom
# 6287760368781 milt

# Mengirim data melalui REST API
url = f'http://localhost:8998/botquestion/{user_number}'
headers = {'Content-type': 'application/json'}
data = json.dumps([{'id': b.id, 
                    'question': b.question, 
                    'answer': b.answer, 
                    'multiplier': b.multiplier, 
                    'score': b.score} for b in botquestion])
print(data)
response = requests.put(url, headers=headers, data=data)

# Menampilkan respons dari server
if response.status_code == 200:
    print('Data berhasil dikirim!')
else:
    print('Terjadi kesalahan saat mengirim data.')

    

[{"id": 0, "question": "Bagaimana cara Anda memotivasi diri sendiri ketika bekerja secara mandiri?", "answer": "", "multiplier": 1, "score": 1}, {"id": 1, "question": "Apakah Anda pernah bekerja secara mandiri sebelumnya? Jika ya, ceritakan pengalaman Anda dan bagaimana Anda menghadapi tantangan tersebut.", "answer": "", "multiplier": 1, "score": 1}, {"id": 2, "question": "Bagaimana Anda mengatur waktu dan prioritas ketika bekerja secara mandiri?", "answer": "", "multiplier": 1, "score": 1}, {"id": 3, "question": "Bagaimana Anda menyelesaikan tugas yang sulit atau kompleks ketika tidak ada orang lain yang bisa membantu?", "answer": "", "multiplier": 1, "score": 1}]
Data berhasil dikirim!


In [11]:
group_number = "120363149813038443@g.us"

In [12]:
print(requests.get(f'http://localhost:8998/getmode/{user_number}').json())

{'message': 'ConvMode.CHITCHAT'}


In [13]:
convmode = ConvMode.ASK
print(requests.get(f'http://localhost:8998/convmode/{user_number}/{convmode}'))

<Response [200]>


## jalankan pertanyaan nomor

In [14]:
#jalankan question 1
#print(requests.get(f'http://localhost:8998/run_question/{user_number}/1'))

In [15]:
script = Script.INTERVIEW
print(requests.get(f'http://localhost:8998/set_script/{user_number}/{script}'))

<Response [200]>


In [16]:
print(requests.get(f'http://localhost:8998/start_question/{user_number}'))

<Response [200]>


In [17]:
print(requests.put(f'http://localhost:8998/interval/{user_number}/600').json())

{'message': 'sudah di set menjadi 600'}


In [18]:
#print(requests.get(f'http://localhost:8998/call_method').json())

In [19]:
print(requests.get(f'http://localhost:8998/botquestions/{user_number}').json())

{'message': 'Bagaimana cara Anda memotivasi diri sendiri ketika bekerja secara mandiri?:\nApakah Anda pernah bekerja secara mandiri sebelumnya? Jika ya, ceritakan pengalaman Anda dan bagaimana Anda menghadapi tantangan tersebut.:\nBagaimana Anda mengatur waktu dan prioritas ketika bekerja secara mandiri?:\nBagaimana Anda menyelesaikan tugas yang sulit atau kompleks ketika tidak ada orang lain yang bisa membantu?:\n'}


In [20]:
print(requests.get(f'http://localhost:8998/obj_info/{user_number}').json())

{'message': {'botquestions': [{'id': 0, 'question': 'Bagaimana cara Anda memotivasi diri sendiri ketika bekerja secara mandiri?', 'answer': '', 'metadata': '', 'koherensi': 1, 'multiplier': 1, 'score': 1}, {'id': 1, 'question': 'Apakah Anda pernah bekerja secara mandiri sebelumnya? Jika ya, ceritakan pengalaman Anda dan bagaimana Anda menghadapi tantangan tersebut.', 'answer': '', 'metadata': '', 'koherensi': 1, 'multiplier': 1, 'score': 1}, {'id': 2, 'question': 'Bagaimana Anda mengatur waktu dan prioritas ketika bekerja secara mandiri?', 'answer': '', 'metadata': '', 'koherensi': 1, 'multiplier': 1, 'score': 1}, {'id': 3, 'question': 'Bagaimana Anda menyelesaikan tugas yang sulit atau kompleks ketika tidak ada orang lain yang bisa membantu?', 'answer': '', 'metadata': '', 'koherensi': 1, 'multiplier': 1, 'score': 1}], 'interval': 600, 'mode': '2', 'wait_time': 0, 'max_token': 4192, 'word_limit': 1000, 'temperature': 0.7}}
